<a href="https://colab.research.google.com/github/pinilDissanayaka/Simple-RAG/blob/main/Simple%20RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install a specific version (2.2.2) of the `sentence-transformers` package
!pip install sentence-transformers==2.2.2

# Install the `faiss-gpu` package for GPU-accelerated similarity search
!pip install faiss-gpu

# Install the `langchain` package, a library for building language model applications
!pip install langchain

# Install the `langchain_community` package, which includes community-contributed extensions for `langchain`
!pip install langchain_community

# Install the `langchain_groq` package, which adds support for Groq-based language model integration
!pip install langchain_groq

# Install the `langchain_huggingface` package to integrate `langchain` with Hugging Face models
!pip install langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 739.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl

In [4]:
# Import the 'os' module for interacting with the operating system
import os

# Import the 'ChatGroq' class from the 'langchain_groq' module, which provides integration with Groq-based language models
from langchain_groq import ChatGroq

# Import the 'WebBaseLoader' class from 'langchain.document_loaders', used to load documents from web sources
from langchain.document_loaders.web_base import WebBaseLoader

# Import the 'HuggingFaceEmbeddings' class from 'langchain_huggingface', which provides embeddings using Hugging Face models
from langchain_huggingface import HuggingFaceEmbeddings

# Import the 'FAISS' class from 'langchain.vectorstores', which is used for vector similarity search with FAISS
from langchain.vectorstores import FAISS

# Import the 'RetrievalQA' class from 'langchain.chains', used to create a QA chain that retrieves relevant documents and answers questions
from langchain.chains import RetrievalQA

# Import the 'ChatPromptTemplate' class from 'langchain.prompts', used to define chat-based prompts for language models
from langchain.prompts import ChatPromptTemplate

# Import the 'RunnablePassthrough' class from 'langchain_core.runnables', which acts as a pass-through runnable for chaining components
from langchain_core.runnables import RunnablePassthrough

# Import the 'StrOutputParser' class from 'langchain_core.output_parsers', used to parse output as strings
from langchain_core.output_parsers import StrOutputParser

In [5]:
# Set an environment variable 'GROQ_API_KEY' using the value obtained from 'userdata'
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')

In [6]:
# Instantiate the 'ChatGroq' class to create a language model object
llm=ChatGroq(model="llama3-70b-8192",
             temperature=0.6)

In [14]:
# Use the 'load' method of 'WebBaseLoader' to fetch and parse the content from the specified URL
loader=WebBaseLoader("https://english.newsfirst.lk/2024/08/13/five-sri-lankans-fighting-for-russia-detained-in-ukraine")
data=loader.load()

print(data)

[Document(metadata={'source': 'https://english.newsfirst.lk/2024/08/13/five-sri-lankans-fighting-for-russia-detained-in-ukraine', 'title': 'Five Sri Lankans Fighting For Russia Detained in Ukraine', 'description': "COLOMBO (News 1st); It has been reported that five Sri Lankan individuals fighting for Russia, have been detained in Ukraine.&nbsp;The Ministry of For - Get the latest breaking news and top stories from Sri Lanka, the latest political news, sports news, weather updates, exam results, business news, entertainment news, world news and much more from News 1st, Sri Lanka's leading news network.", 'language': 'en'}, page_content="\n\nFive Sri Lankans Fighting For Russia Detained in Ukraine\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nhomeHomeLatestFeaturedLocalSportsBusinessWorldEnglishsearchEnglishmenuFive Sri Lankans Fighting For Russia Detained in Ukraine by Staff Writer 13-08-2024 | 11:23 AMCOLOMBO (News 1st); It has been reported that five Sri Lankan individuals figh

In [15]:
# Instantiate the 'HuggingFaceEmbeddings' class to create an embeddings model object
embeddingModel = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [16]:
# Create a vector database using the 'FAISS' class
# 'FAISS.from_documents' initializes the FAISS vector store from a list of documents
vectorDAtabase=FAISS.from_documents(data, embeddingModel)

In [17]:
# 'vectorDatabase.as_retriever()' creates a retriever that can be used to query the vector database
retriever=vectorDAtabase.as_retriever()

In [18]:
# Define a prompt template as a multi-line string
# This template is used to generate a prompt for the language model

template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


chatPromptTemplate = ChatPromptTemplate.from_template(template)

In [19]:
# Create a chain of components that processes a question based on the provided context

chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | chatPromptTemplate
    | llm
    | StrOutputParser()
)

In [20]:
# Invoke the processing chain with a specific question

chain.invoke("How many  Sri Lankans fighting For russia detained in Ukraine")

'According to the context, "It has been reported that five Sri Lankan individuals fighting for Russia, have been detained in Ukraine."'